In [1]:
conda install -c conda-forge folium


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.9.2



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [12]:
import sys
!{sys.executable} -m pip install wget

  Using cached wget-3.2.zip (10 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=0711048778a6c28da108e2183e1592e636e66c2b0123d86181282de96881f7ad
  Stored in directory: /Users/berkedenizbozyigit/Library/Caches/pip/wheels/46/78/0e/8e5e2b500f83a682c8d7e7ce820638cf99faa894a662f71cf0
Successfully built wget


In [13]:
import folium
import wget
import pandas as pd

In [14]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [15]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [............................................................] 7710 / 7710

In [16]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [17]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [18]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [28]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    
    coordinate = [row['Lat'], row['Long']]
    
    label = row['Launch Site']
    
    folium.Circle(
        location = coordinate,
        radius = 1000,
        color='#000000',
        fill=True,
        fill_color='#3186cc'
    
    ).add_child(folium.Popup(label)).add_to(site_map)
    
    #Add marker for each launch site.
    
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{label}</b></div>',
        )
    ).add_to(site_map)
        

site_map        

In [26]:
print(launch_sites_df.columns)

Index(['Launch Site', 'Lat', 'Long'], dtype='object')


In [29]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [30]:
marker_cluster = MarkerCluster()

In [34]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [35]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [36]:
from folium.plugins import MarkerCluster

In [38]:
marker_cluster = MarkerCluster()

site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    
    coordinate = [record['Lat'], record['Long']]
    
    color = record['marker_color']
    
    marker = folium.Marker(
    location=coordinate,
    icon=folium.Icon(color='white', icon_color=color))
    
    marker_cluster.add_child(marker)
    
site_map    

In [39]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [40]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [41]:
from folium.plugins import MousePosition

In [47]:
# Initialize the map
site_map = folium.Map(location=[row['Lat'], row['Long']], zoom_start=10)

# Add MousePosition plugin to display coordinates on the map
MousePosition().add_to(site_map)

# Display the map to manually find the closest coastline coordinate
site_map

In [46]:
print(launch_sites_df[launch_sites_df['Lat'].apply(lambda x: isinstance(x, list)) | 
                   launch_sites_df['Long'].apply(lambda x: isinstance(x, list))])

Empty DataFrame
Columns: [Launch Site, Lat, Long]
Index: []


In [52]:
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance_km = 6371 * c  # Radius of Earth in kilometers
    return distance_km

# Replace coastline_lat and coastline_lon with the noted coastline coordinates
coastline_lat = 28.56367  # Example
coastline_lon = -80.57163  # Example

# Calculate the distance
distance_coastline = calculate_distance(row['Lat'], row['Long'], coastline_lat, coastline_lon)
print(f"Distance to the coastline: {distance_coastline:.2f} km")




Distance to the coastline: 3826.31 km


In [56]:
distance = calculate_distance(row['Lat'], row['Long'], coastline_lat, coastline_lon)


# Add a marker for the launch site
folium.Marker(
    [row['Lat'], row['Long']],
    icon=folium.Icon(color="blue", icon="info-sign"),
    popup="Launch Site"
).add_to(site_map)

# Add a marker for the closest coastline point with the distance displayed
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance),
    ),
    popup="Closest Coastline Point"
)
distance_marker.add_to(site_map)

# Display the map
site_map

In [57]:
# Create a PolyLine to connect the launch site and the coastline point
coordinates = [[row['Lat'], row['Long']], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(lines)

# Display the map
site_map

In [60]:
railway_lat, railway_lon = 28.564, -80.57  # Replace with closest railway coordinates
highway_lat, highway_lon = 28.565, -80.575  # Replace with closest highway coordinates
city_lat, city_lon = 28.61, -80.62  # Replace with closest city coordinates


# Draw PolyLine to the railway
railway_line = folium.PolyLine(locations=[[row['Lat'], row['Long']], [railway_lat, railway_lon]], weight=1, color='green')
site_map.add_child(railway_line)

# Draw PolyLine to the highway
highway_line = folium.PolyLine(locations=[[row['Lat'], row['Long']], [highway_lat, highway_lon]], weight=1, color='orange')
site_map.add_child(highway_line)

# Draw PolyLine to the city
city_line = folium.PolyLine(locations=[[row['Lat'], row['Long']], [city_lat, city_lon]], weight=1, color='purple')
site_map.add_child(city_line)

# Display the map
site_map